In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt

import remap_values as rv

import cPickle

In [2]:
from sklearn.utils import shuffle
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.svm import SVR
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.base import clone

In [3]:
inp_df = pd.read_csv( "inp_data/train.csv" )

my_df      = rv.run_clean( inp_df, train=True )
my_df      = shuffle( my_df, random_state=0 )

feature_df = my_df.drop( ['SalePrice'], axis=1 ).copy()
label_df   = my_df['SalePrice'].copy()

In [4]:
test_df    = pd.read_csv( "inp_data/test.csv" )
test_df    = rv.run_clean( test_df )

In [5]:
test_df['HouseStyle_5'] = 0

train_cols = feature_df.columns
test_df    = test_df[ train_cols ].copy()

In [6]:
from sklearn.linear_model import Ridge # alpha 1
from sklearn.linear_model import Lasso # alpha 1
from sklearn.linear_model import BayesianRidge # alpha_1, alpha_2, lambda_1, lambda_2, both def 1e-6
from sklearn.linear_model import SGDRegressor
from sklearn.kernel_ridge import KernelRidge # alpha , kernel, gamma
from sklearn.ensemble     import GradientBoostingRegressor

In [9]:
gen_params = {'kernel':['rbf'], 
              'C':      [5e-1,1e0,5e0],
              'gamma':  [5e-3,1e-2,5e-2],
              'epsilon':[5e-3,1e-2,5e-2]}
svr_rbf_reg = rv.optimize_fit( SVR(), feature_df.values, label_df.values, gen_params, nf=10 )

Fold  1 accuracy: 0.8835  ,  0.8644  {'epsilon': 0.01, 'C': 5.0, 'gamma': 0.005, 'kernel': 'rbf'}
Fold  2 accuracy: 0.7903  ,  0.8720  {'epsilon': 0.01, 'C': 5.0, 'gamma': 0.005, 'kernel': 'rbf'}
Fold  3 accuracy: 0.9170  ,  0.8594  {'epsilon': 0.01, 'C': 5.0, 'gamma': 0.005, 'kernel': 'rbf'}
Fold  4 accuracy: 0.8778  ,  0.8647  {'epsilon': 0.01, 'C': 5.0, 'gamma': 0.005, 'kernel': 'rbf'}
Fold  5 accuracy: 0.8198  ,  0.8682  {'epsilon': 0.01, 'C': 5.0, 'gamma': 0.005, 'kernel': 'rbf'}
Fold  6 accuracy: 0.8850  ,  0.8654  {'epsilon': 0.01, 'C': 5.0, 'gamma': 0.005, 'kernel': 'rbf'}
Fold  7 accuracy: 0.8839  ,  0.8649  {'epsilon': 0.01, 'C': 5.0, 'gamma': 0.005, 'kernel': 'rbf'}
Fold  8 accuracy: 0.8822  ,  0.8649  {'epsilon': 0.01, 'C': 5.0, 'gamma': 0.005, 'kernel': 'rbf'}
Fold  9 accuracy: 0.9074  ,  0.8602  {'epsilon': 0.01, 'C': 5.0, 'gamma': 0.005, 'kernel': 'rbf'}
Fold 10 accuracy: 0.8696  ,  0.8670  {'epsilon': 0.01, 'C': 5.0, 'gamma': 0.005, 'kernel': 'rbf'}
 
Clf  0 Mean Accura

In [11]:
gen_params = {'kernel': ['poly'], 
              'C':      [5e0,1e1,5e1],
              'degree': [3],
              'epsilon':[5e-3,1e-2,5e-2]}
svr_pol_reg = rv.optimize_fit( SVR(), feature_df.values, label_df.values, gen_params, nf=10 )

Fold  1 accuracy: 0.8774  ,  0.8395  {'epsilon': 0.005, 'C': 10.0, 'degree': 3, 'kernel': 'poly'}
Fold  2 accuracy: 0.8056  ,  0.8436  {'epsilon': 0.005, 'C': 10.0, 'degree': 3, 'kernel': 'poly'}
Fold  3 accuracy: 0.9082  ,  0.8326  {'epsilon': 0.01, 'C': 10.0, 'degree': 3, 'kernel': 'poly'}
Fold  4 accuracy: 0.8512  ,  0.8419  {'epsilon': 0.01, 'C': 50.0, 'degree': 3, 'kernel': 'poly'}
Fold  5 accuracy: 0.7780  ,  0.8459  {'epsilon': 0.01, 'C': 10.0, 'degree': 3, 'kernel': 'poly'}
Fold  6 accuracy: 0.8522  ,  0.8433  {'epsilon': 0.01, 'C': 10.0, 'degree': 3, 'kernel': 'poly'}
Fold  7 accuracy: 0.8660  ,  0.8399  {'epsilon': 0.01, 'C': 10.0, 'degree': 3, 'kernel': 'poly'}
Fold  8 accuracy: 0.8523  ,  0.8405  {'epsilon': 0.01, 'C': 50.0, 'degree': 3, 'kernel': 'poly'}
Fold  9 accuracy: 0.9002  ,  0.8362  {'epsilon': 0.01, 'C': 50.0, 'degree': 3, 'kernel': 'poly'}
Fold 10 accuracy: 0.8397  ,  0.8448  {'epsilon': 0.01, 'C': 50.0, 'degree': 3, 'kernel': 'poly'}
 
Clf  0 Mean Accuracy: 0.85

In [13]:
gen_params = {'alpha': np.arange(0.1,2.0,0.01),
              'fit_intercept':[True]}
ridge_reg = rv.optimize_fit( Ridge(), feature_df.values, label_df.values, gen_params, nf=10 ) 

Fold  1 accuracy: 0.8602  ,  0.8592  {'alpha': 0.63999999999999968, 'fit_intercept': True}
Fold  2 accuracy: 0.7769  ,  0.8695  {'alpha': 0.35999999999999988, 'fit_intercept': True}
Fold  3 accuracy: 0.9092  ,  0.8527  {'alpha': 1.0299999999999996, 'fit_intercept': True}
Fold  4 accuracy: 0.8621  ,  0.8572  {'alpha': 1.1999999999999995, 'fit_intercept': True}
Fold  5 accuracy: 0.8348  ,  0.8635  {'alpha': 1.0799999999999996, 'fit_intercept': True}
Fold  6 accuracy: 0.8862  ,  0.8595  {'alpha': 1.1199999999999997, 'fit_intercept': True}
Fold  7 accuracy: 0.8686  ,  0.8638  {'alpha': 0.6599999999999997, 'fit_intercept': True}
Fold  8 accuracy: 0.8761  ,  0.8616  {'alpha': 0.55999999999999972, 'fit_intercept': True}
Fold  9 accuracy: 0.8954  ,  0.8580  {'alpha': 0.37999999999999989, 'fit_intercept': True}
Fold 10 accuracy: 0.8753  ,  0.8636  {'alpha': 0.59999999999999976, 'fit_intercept': True}
 
Clf  0 Mean Accuracy: 0.8649 +/- 0.0344
Clf  1 Mean Accuracy: 0.8646 +/- 0.0351
Clf  2 Mean A

In [14]:
gen_params = {'alpha': 10**-np.arange(1.0,5.0,0.5),
              'fit_intercept':[True,False]}
Lasso_reg = rv.optimize_fit( Lasso(), feature_df.values, label_df.values, gen_params, nf=10 ) 

Fold  1 accuracy: 0.8725  ,  0.8637  {'alpha': 0.00031622776601683794, 'fit_intercept': True}
Fold  2 accuracy: 0.7884  ,  0.8728  {'alpha': 0.0001, 'fit_intercept': True}
Fold  3 accuracy: 0.9114  ,  0.8570  {'alpha': 0.00031622776601683794, 'fit_intercept': True}
Fold  4 accuracy: 0.8616  ,  0.8635  {'alpha': 0.00031622776601683794, 'fit_intercept': True}
Fold  5 accuracy: 0.8213  ,  0.8697  {'alpha': 0.00031622776601683794, 'fit_intercept': True}
Fold  6 accuracy: 0.8829  ,  0.8644  {'alpha': 0.00031622776601683794, 'fit_intercept': True}
Fold  7 accuracy: 0.8652  ,  0.8678  {'alpha': 0.0001, 'fit_intercept': False}
Fold  8 accuracy: 0.8843  ,  0.8640  {'alpha': 0.0001, 'fit_intercept': True}
Fold  9 accuracy: 0.8955  ,  0.8597  {'alpha': 0.0001, 'fit_intercept': True}
Fold 10 accuracy: 0.8761  ,  0.8669  {'alpha': 0.0001, 'fit_intercept': True}
 
Clf  0 Mean Accuracy: 0.8665 +/- 0.0313
Clf  1 Mean Accuracy: 0.8676 +/- 0.0328
Clf  2 Mean Accuracy: 0.8665 +/- 0.0313
Clf  3 Mean Accur

In [15]:
gen_params = {'alpha_1':  10**np.arange( -3.0,-1.0, 1.0 ),
              'alpha_2':  10**np.arange( -3.0,-1.0, 1.0 ),
              'lambda_1': 10**np.arange( -3.0,-1.0, 1.0 ),
              'lambda_2': 10**np.arange( -3.0,-1.0, 1.0 )}
br_reg = rv.optimize_fit( BayesianRidge(), feature_df.values, label_df.values, gen_params, nf=10 ) 

Fold  1 accuracy: 0.8625  ,  0.8580  {'alpha_2': 0.001, 'lambda_1': 0.001, 'lambda_2': 0.01, 'alpha_1': 0.01}
Fold  2 accuracy: 0.7842  ,  0.8682  {'alpha_2': 0.001, 'lambda_1': 0.001, 'lambda_2': 0.01, 'alpha_1': 0.01}
Fold  3 accuracy: 0.9083  ,  0.8518  {'alpha_2': 0.001, 'lambda_1': 0.001, 'lambda_2': 0.01, 'alpha_1': 0.01}
Fold  4 accuracy: 0.8617  ,  0.8566  {'alpha_2': 0.001, 'lambda_1': 0.001, 'lambda_2': 0.01, 'alpha_1': 0.01}
Fold  5 accuracy: 0.8319  ,  0.8627  {'alpha_2': 0.001, 'lambda_1': 0.001, 'lambda_2': 0.01, 'alpha_1': 0.01}
Fold  6 accuracy: 0.8849  ,  0.8588  {'alpha_2': 0.001, 'lambda_1': 0.001, 'lambda_2': 0.01, 'alpha_1': 0.01}
Fold  7 accuracy: 0.8689  ,  0.8621  {'alpha_2': 0.001, 'lambda_1': 0.001, 'lambda_2': 0.01, 'alpha_1': 0.01}
Fold  8 accuracy: 0.8749  ,  0.8594  {'alpha_2': 0.001, 'lambda_1': 0.001, 'lambda_2': 0.01, 'alpha_1': 0.01}
Fold  9 accuracy: 0.8958  ,  0.8552  {'alpha_2': 0.001, 'lambda_1': 0.001, 'lambda_2': 0.01, 'alpha_1': 0.01}
Fold 10 ac

In [20]:
gen_params = {'kernel':['rbf'],
              'alpha':10**np.arange( -4, -3, 0.2),
              'gamma':10**np.arange( -4, -3, 0.2)}
kRBF_reg = rv.optimize_fit( KernelRidge(), feature_df.values, label_df.values, gen_params, nf=10 ) 

Fold  1 accuracy: 0.8671  ,  0.8637  {'alpha': 0.00039810717055349773, 'gamma': 0.00063095734448019429, 'kernel': 'rbf'}
Fold  2 accuracy: 0.7898  ,  0.8724  {'alpha': 0.0001, 'gamma': 0.00025118864315095823, 'kernel': 'rbf'}
Fold  3 accuracy: 0.9158  ,  0.8564  {'alpha': 0.00063095734448019429, 'gamma': 0.00063095734448019429, 'kernel': 'rbf'}
Fold  4 accuracy: 0.8693  ,  0.8617  {'alpha': 0.00063095734448019429, 'gamma': 0.00063095734448019429, 'kernel': 'rbf'}
Fold  5 accuracy: 0.8385  ,  0.8674  {'alpha': 0.00063095734448019429, 'gamma': 0.00063095734448019429, 'kernel': 'rbf'}
Fold  6 accuracy: 0.8910  ,  0.8640  {'alpha': 0.00063095734448019429, 'gamma': 0.00063095734448019429, 'kernel': 'rbf'}
Fold  7 accuracy: 0.8734  ,  0.8675  {'alpha': 0.00063095734448019429, 'gamma': 0.00063095734448019429, 'kernel': 'rbf'}
Fold  8 accuracy: 0.8780  ,  0.8653  {'alpha': 0.00039810717055349773, 'gamma': 0.00063095734448019429, 'kernel': 'rbf'}
Fold  9 accuracy: 0.9031  ,  0.8619  {'alpha': 0

In [22]:
gen_params = {'kernel':['poly'],
              'degree':[3],
              'gamma':10**np.arange( -3,-1, 0.3)}
kPol_reg = rv.optimize_fit( KernelRidge(), feature_df.values, label_df.values, gen_params, nf=10 ) 

Fold  1 accuracy: 0.8418  ,  0.8149  {'kernel': 'poly', 'gamma': 0.0079432823472428051, 'degree': 3}
Fold  2 accuracy: 0.7872  ,  0.8202  {'kernel': 'poly', 'gamma': 0.0079432823472428051, 'degree': 3}
Fold  3 accuracy: 0.8845  ,  0.8085  {'kernel': 'poly', 'gamma': 0.0079432823472428051, 'degree': 3}
Fold  4 accuracy: 0.8024  ,  0.8174  {'kernel': 'poly', 'gamma': 0.0079432823472428051, 'degree': 3}
Fold  5 accuracy: 0.8063  ,  0.8216  {'kernel': 'poly', 'gamma': 0.0079432823472428051, 'degree': 3}
Fold  6 accuracy: 0.8465  ,  0.8164  {'kernel': 'poly', 'gamma': 0.0079432823472428051, 'degree': 3}
Fold  7 accuracy: 0.8292  ,  0.8186  {'kernel': 'poly', 'gamma': 0.0079432823472428051, 'degree': 3}
Fold  8 accuracy: 0.8355  ,  0.8148  {'kernel': 'poly', 'gamma': 0.0079432823472428051, 'degree': 3}
Fold  9 accuracy: 0.8723  ,  0.8117  {'kernel': 'poly', 'gamma': 0.0079432823472428051, 'degree': 3}
Fold 10 accuracy: 0.8304  ,  0.8189  {'kernel': 'poly', 'gamma': 0.0079432823472428051, 'de

In [23]:
gen_params = {'loss':['huber'],
              'n_estimators':[300,400],
              'max_depth':[2,4,6],
              'min_samples_leaf':[3,5,8],
              'alpha':[0.8,0.85,0.9,0.95]}
boost_reg = rv.optimize_fit( GradientBoostingRegressor(), feature_df.values, label_df.values, gen_params, nf = 10 )

Fold  1 accuracy: 0.8244  ,  0.8748  {'n_estimators': 300, 'alpha': 0.8, 'min_samples_leaf': 3, 'max_depth': 4, 'loss': 'huber'}
Fold  2 accuracy: 0.7998  ,  0.8743  {'n_estimators': 400, 'alpha': 0.9, 'min_samples_leaf': 8, 'max_depth': 2, 'loss': 'huber'}
Fold  3 accuracy: 0.9180  ,  0.8663  {'n_estimators': 300, 'alpha': 0.8, 'min_samples_leaf': 3, 'max_depth': 4, 'loss': 'huber'}
Fold  4 accuracy: 0.8823  ,  0.8701  {'n_estimators': 300, 'alpha': 0.85, 'min_samples_leaf': 3, 'max_depth': 4, 'loss': 'huber'}
Fold  5 accuracy: 0.8376  ,  0.8697  {'n_estimators': 300, 'alpha': 0.9, 'min_samples_leaf': 3, 'max_depth': 2, 'loss': 'huber'}
Fold  6 accuracy: 0.8891  ,  0.8719  {'n_estimators': 400, 'alpha': 0.95, 'min_samples_leaf': 3, 'max_depth': 2, 'loss': 'huber'}
Fold  7 accuracy: 0.8635  ,  0.8710  {'n_estimators': 300, 'alpha': 0.85, 'min_samples_leaf': 3, 'max_depth': 4, 'loss': 'huber'}
Fold  8 accuracy: 0.9123  ,  0.8650  {'n_estimators': 400, 'alpha': 0.85, 'min_samples_leaf': 

In [ ]:
#svr_rbf_red
#boost_reg

In [24]:
with open('svr_rbf_reg3.pkl','wb') as fid:
    cPickle.dump( svr_rbf_reg, fid )
with open('boost_reg3.pkl','wb') as fid:
    cPickle.dump( boost_reg, fid )

In [6]:
with open('svr_rbf_reg2.pkl','r') as fid:
    svr_rbf_reg = cPickle.load( fid )
with open('boost_reg2.pkl','r') as fid:
    boost_reg   = cPickle.load( fid )

In [30]:
lin_pred = 10**(   boost_reg.predict( test_df ) )
rbf_pred = 10**( svr_rbf_reg.predict( test_df ) )

In [32]:
id_df  = pd.read_csv( "inp_data/test.csv" )[['Id','Fence']]

lin_df = id_df.copy()
rbf_df = id_df.copy()

lin_df['SalePrice'] = lin_pred
rbf_df['SalePrice'] = rbf_pred

lin_df = lin_df.drop( ['Fence'], axis=1 )
rbf_df = rbf_df.drop( ['Fence'], axis=1 )

In [34]:
lin_df.to_csv( 'boost_3.csv', index=False )
rbf_df.to_csv( 'rbf_3.csv'  , index=False )

In [6]:
with open('svr_lin.pkl', 'rb') as fid:
    svr_lin = cPickle.load(fid)
with open('svr_rbf.pkl', 'rb') as fid:
    svr_rbf = cPickle.load(fid)

In [26]:
lin_pred = (   boost_reg.predict( feature_df ) )
rbf_pred = ( svr_rbf_reg.predict( feature_df ) )

In [27]:
lin_diff = label_df.values - lin_pred
rbf_diff = label_df.values - rbf_pred

In [28]:
my_df['LinPred'] = lin_pred
my_df['RBFPred'] = rbf_pred
my_df['LinDiff'] = lin_diff
my_df['RBFDiff'] = rbf_diff

In [29]:
preds = ['LinPred','RBFPred']
for pred in preds:
    my_df.plot(kind='scatter',x='SalePrice',y='LinPred')
    plt.show()

/home/sean/anaconda/lib/python2.7/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


In [46]:
outliers = my_df[ abs(my_df['RBFDiff']) >0.09 ].copy()
#outliers =    my_df[    my_df['Electrical'] == 0 ].copy()
#outliers = outliers[ outliers['CentralAir'] == 0 ]
#outliers = outliers[ outliers['SaleCondition'] != 0 ]
#outliers = outliers[ outliers['MSZoning'  ] != 2./3 ]
#outliers = outliers[ outliers['MSZoning'  ] != 2./3 ]
#outliers = outliers[ outliers['MSZoning'  ] != 2./3 ]

In [51]:
cols = my_df.drop( ['InsideSF','OutsideSF','GarageArea','LotArea', 'LotFrontage',
                    'SalePrice', 'RBFPred'], axis=1 ).columns.values
#sns.lmplot( 'SalePrice', 'RBFPred', data=my_df, hue='LotFrontage', fit_reg = False)#, palette="rainbow" )

lowScat = ['CentralAir', 'Electrical','SaleCondition',
           'InsideSF','OutsideSF', 'LotArea',
           'RBFPred','RBFDiff','SalePrice']
scat_df = outliers[lowScat].copy()


for col in lowScat[:-3]:
    g = sns.lmplot( 'SalePrice', 'RBFPred', data=scat_df, hue=col, fit_reg = False, palette="rainbow" )
    g.fig.suptitle( col )
    plt.plot( [4,6], [4,6], color='b')
    plt.xlim( 4.3,5.7 )
    plt.ylim( 4.3,5.7 )
    plt.show()
#for col in cols:
#    g = sns.lmplot( 'SalePrice', 'RBFPred', data=my_df, hue=col, fit_reg=False)
#    g.fig.suptitle( col )
#    plt.show()
#    plt.clf()

In [38]:
for col in lowScat[:-2]:
    scat_df[col].plot( kind='hist', title=col, bins=50 )
    plt.show()

In [ ]:
#0.051, 0.091,, 0.15, 0.2
#Zoning 3 or 4? whichever makes 0.6..., 0.6, 0.6
#CentralAir 1? 1, 0, lots of 0
#Exterior1st 0? 0, 0, 0
#Electrical 0? 0, 0, 0
#SaleCOnd 0? 0/1/5, 1, 1/5